In [15]:
import os
import random
import numpy as np
import pandas as pd
import cudf, itertools
import scipy.sparse as ssp
from functools import lru_cache
from tqdm import tqdm, trange
from collections import Counter, defaultdict

In [16]:
merged_candidates_feature_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_feature_2.parquet'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv'
train_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_train_sessions.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'

In [17]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature():
    return pd.read_parquet(merged_candidates_feature_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_train_sessions():
    return pd.read_csv(train_sessions_path)

In [18]:
def cast_dtype(df : pd.DataFrame):
    for k in df.columns:
        dt = type(df[k].iloc[0])
        if 'float' in str(dt):
            df[k] = df[k].astype('float32')
        elif 'int' in str(dt):
            df[k] = df[k].astype('int32')
        elif dt == list:
            dt_ = type(df.iloc[0][k][0])
            if 'float' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.float32))
            elif 'int' in str(dt_):
                df[k] = df[k].apply(lambda x : np.array(x, dtype=np.int32))

In [19]:
def cal_item_freq(item_counter:Counter, session_df:pd.DataFrame, test=False):
    for i in tqdm(range(session_df.shape[0])):
        sess = session_df.iloc[i]
        prev_items = eval(sess['prev_items'].replace(' ', ','))
        for item in prev_items:
            item_counter[item] += 1
        if not test:
            next_item = sess['next_item'] 
            item_counter[next_item] += 1 

# Merge valid item frequency

In [20]:
valid_sessions_df = read_valid_sessions()
train_sessions_df = read_train_sessions()

In [21]:
merged_candidates_feature = read_merged_candidates_feature()

In [22]:
item_counter = Counter()
cal_item_freq(item_counter, train_sessions_df, test=False)
cal_item_freq(item_counter, valid_sessions_df, test=True)

100%|██████████| 361581/361581 [00:16<00:00, 21631.55it/s]


In [23]:
item_counter.most_common(10)

[('B07QPV9Z7X', 2993),
 ('B0BD5MFPMF', 2851),
 ('B01MXLEVR7', 2383),
 ('B09NQGVSPD', 2294),
 ('B08CN3G4N9', 2271),
 ('B08GWS298V', 2234),
 ('B00NTCH52W', 2202),
 ('B0BDML9477', 2156),
 ('B0BD88WWQ8', 2106),
 ('B07RHT52HX', 2101)]

In [24]:
products, counts = zip(*item_counter.items())
item_freq_df = pd.DataFrame({'product' : products, 'product_freq' : counts})

In [25]:
item_freq_df_g = cudf.from_pandas(item_freq_df)
merged_candidates_feature_g = cudf.from_pandas(merged_candidates_feature)

In [26]:
merged_candidates_freq_g = merged_candidates_feature_g.merge(item_freq_df_g, how='left', left_on=['product'], right_on=['product'])
merged_candidates_freq_g = merged_candidates_freq_g.sort_values(by=['sess_id']).reset_index(drop=True)
merged_candidates_freq_g['product_freq'] = merged_candidates_freq_g['product_freq'].fillna(0)
cast_dtype(merged_candidates_freq_g)

In [27]:
merged_candidates_freq = merged_candidates_freq_g.to_pandas()
merged_candidates_freq.to_parquet(merged_candidates_feature_path, engine='pyarrow')

In [28]:
merged_candidates_freq

,sess_id,sess_locale,product,target,sasrec_normalized_scores,roberta_normalized_scores,co_graph_normalized_counts,co_graph_normalized_counts_0,co_graph_normalized_counts_1,co_graph_normalized_counts_2,product_freq
0,0,UK,B07TBQB8FK,0.0,0.000000,0.000000,0.005037,0.006873,0.005402,0.003190,43
1,0,UK,B01MSZE4YK,0.0,0.000000,0.000000,0.001717,0.001718,0.002026,0.001595,23
2,0,UK,B07VC5C7Z7,0.0,0.000000,0.000000,0.003778,0.003436,0.006077,0.003190,173
3,0,UK,B0856KQ4D2,0.0,0.000276,0.007527,0.000000,0.000000,0.000000,0.000000,81
4,0,UK,B01HF2A8PC,0.0,0.000406,0.019186,0.000000,0.000000,0.000000,0.000000,5
...,...,...,...,...,...,...,...,...,...,...,...
85447871,361580,DE,B09D7RH5QG,0.0,0.000880,0.003712,0.000000,0.000000,0.000000,0.000000,17
85447872,361580,DE,B07CJJVQRJ,0.0,0.000013,0.000000,0.000000,0.000000,0.000000,0.000000,10
85447873,361580,DE,B09W4WL5BN,0.0,0.000000,0.004873,0.000000,0.000000,0.000000,0.000000,11
85447874,361580,DE,B08D9PGC9P,0.0,0.000071,0.014974,0.000000,0.000000,0.000000,0.000000,128


# Merge test item frequency

In [9]:
merged_candidates_feature_test_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_feature_test.parquet'
valid_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_valid_sessions.csv'
train_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/data_for_recstudio/task1_data/task13_4_task1_train_sessions.csv'
test_sessions_path = '/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/raw_data/sessions_test_task1.csv'

In [10]:
@lru_cache(maxsize=1)
def read_merged_candidates_feature_test():
    return pd.read_parquet(merged_candidates_feature_test_path, engine='pyarrow')

@lru_cache(maxsize=1)
def read_valid_sessions():
    return pd.read_csv(valid_sessions_path)

@lru_cache(maxsize=1)
def read_train_sessions():
    return pd.read_csv(train_sessions_path)

@lru_cache(maxsize=1)
def read_test_sessions():
    return pd.read_csv(test_sessions_path)

In [11]:
merged_candidates_feature_test = read_merged_candidates_feature_test()
valid_sessions_df = read_valid_sessions()
train_sessions_df = read_train_sessions()
test_sessions_df = read_test_sessions()

In [79]:
# test sessions are included in train and valid sessions
item_counter = Counter()
cal_item_freq(item_counter, train_sessions_df, test=False)
# cal_item_freq(item_counter, valid_sessions_df, test=True)
cal_item_freq(item_counter, valid_sessions_df, test=False)


100%|██████████| 361581/361581 [00:17<00:00, 20139.22it/s]


In [80]:
products, counts = zip(*item_counter.items())
item_freq_df = pd.DataFrame({'product' : products, 'product_freq' : counts})

In [81]:
merged_candidates_feature_test.drop(columns=['product_freq'], inplace=True)

In [82]:
item_freq_df_g = cudf.from_pandas(item_freq_df)
merged_candidates_feature_g = cudf.from_pandas(merged_candidates_feature_test)

In [ ]:
merged_candidates_feature_g

In [84]:
merged_candidates_freq_g = merged_candidates_feature_g.merge(item_freq_df_g, how='left', left_on=['product'], right_on=['product'])
merged_candidates_freq_g = merged_candidates_freq_g.sort_values(by=['sess_id']).reset_index(drop=True)
merged_candidates_freq_g['product_freq'] = merged_candidates_freq_g['product_freq'].fillna(0)
cast_dtype(merged_candidates_freq_g)

In [85]:
merged_candidates_freq = merged_candidates_freq_g.to_pandas()
merged_candidates_freq.to_parquet(merged_candidates_feature_test_path, engine='pyarrow')

# candidates statistic

In [65]:
merged_candidates_feature_test = pd.read_parquet('/root/autodl-tmp/xiaolong/WorkSpace/Amazon-KDDCUP-23/XGBoost/candidates/merged_candidates_feature_test_2.parquet', engine='pyarrow')

In [66]:
merged_candidates_feature_test

,sess_id,sess_locale,product,sasrec_normalized_scores,roberta_normalized_scores,co_graph_normalized_counts,co_graph_normalized_counts_0,co_graph_normalized_counts_1,co_graph_normalized_counts_2,product_freq
0,0,DE,B09T2ZXL4V,0.000357,0.008733,0.005862,0.012821,0.000000,0.000000,118
1,0,DE,B08QYYBTMC,0.002867,0.000000,0.012702,0.012821,0.007380,0.014286,574
2,0,DE,B0B7S7LBMB,0.000042,0.000000,0.019052,0.032051,0.011070,0.007143,551
3,0,DE,B0BD7MGXMM,0.000378,0.000000,0.002931,0.006410,0.000000,0.000000,101
4,0,DE,B000H6W2GW,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,873
...,...,...,...,...,...,...,...,...,...,...
70403346,316970,UK,B0BGMDVM7V,0.000011,0.014775,0.000000,0.000000,0.000000,0.000000,26
70403347,316970,UK,B09FSZNRD9,0.000000,0.007841,0.000000,0.000000,0.000000,0.000000,20
70403348,316970,UK,B0B7838HH6,0.000000,0.000000,0.038889,0.058824,0.033333,0.030303,169
70403349,316970,UK,B08TJRVWV1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,655


In [67]:
(merged_candidates_feature_test['roberta_normalized_scores'] == 0.0).sum() / 70403351.0

0.5497605930717702

In [68]:
(merged_candidates_feature_test['sasrec_normalized_scores'] == 0.0).sum() / 70403351.0

0.549195776206732

In [69]:
(merged_candidates_feature_test['co_graph_normalized_counts_0'] == 0.0).sum() / 70403351.0

0.6944105402028378

In [70]:
(merged_candidates_feature_test['co_graph_normalized_counts_1'] == 0.0).sum() / 70403351.0

0.7542271389894495

In [71]:
merged_candidates_feature = read_merged_candidates_feature()

In [72]:
(merged_candidates_feature['roberta_normalized_scores'] == 0.0).sum() / 85508299.0

0.57642176930686

In [73]:
(merged_candidates_feature['sasrec_normalized_scores'] == 0.0).sum() / 85508299.0

0.5760398531609195

In [74]:
(merged_candidates_feature['co_graph_normalized_counts_0'] == 0.0).sum() / 85508299.0

0.7768674242952722

In [75]:
(merged_candidates_feature['co_graph_normalized_counts_1'] == 0.0).sum() / 85508299.0

0.8249585341418147

In [15]:
merged_candidates_feature[merged_candidates_feature['product'] == 'B09P2YPF3H']

,sess_id,sess_locale,product,target,sasrec_normalized_scores,roberta_normalized_scores,co_graph_normalized_counts,co_graph_normalized_counts_0,co_graph_normalized_counts_1,co_graph_normalized_counts_2,product_freq
155143,649,UK,B09P2YPF3H,0.0,0.0,0.008176,0.000000,0.000000,0.0,0.0,7
306835,1287,UK,B09P2YPF3H,0.0,0.0,0.007995,0.000000,0.000000,0.0,0.0,7
335697,1407,UK,B09P2YPF3H,0.0,0.0,0.000000,0.014634,0.028571,0.0,0.0,7
353023,1482,UK,B09P2YPF3H,0.0,0.0,0.009932,0.027650,0.064516,0.0,0.0,7
373576,1568,UK,B09P2YPF3H,0.0,0.0,0.009154,0.000000,0.000000,0.0,0.0,7
...,...,...,...,...,...,...,...,...,...,...,...
85008145,358834,UK,B09P2YPF3H,0.0,0.0,0.005578,0.000000,0.000000,0.0,0.0,7
85240941,359819,UK,B09P2YPF3H,0.0,0.0,0.007549,0.000000,0.000000,0.0,0.0,7
85330404,360195,UK,B09P2YPF3H,0.0,0.0,0.010609,0.000000,0.000000,0.0,0.0,7
85332443,360203,UK,B09P2YPF3H,0.0,0.0,0.007566,0.000000,0.000000,0.0,0.0,7
